# Cheat sheet : Practice on Pandas - winemag

A Python notebook to practive the Pandas concepts of the course, mainly on the winemag database.
This code is handwrited, and taken from the Kaggle Pandas course.
It can be used as a cheatsheet, to remember how to use certain Pandas functions, and some examples of what can be done with Pandas.

# Creating, Reading and Writing

In [ ]:
import pandas as pd

reviews = pd.read_csv('data/winemag-data-130k-v2.csv')

# Indexing, Selecting & Assigning

In [ ]:
reviews.head()

In [ ]:
# Select the `description` column from `reviews` and assign the result to the variable `desc`.
desc = reviews.description

In [ ]:
# Select the first value from the description column of reviews, assigning it to variable first_description.
first_description = desc[0]

In [ ]:
# Select the first row of data (the first record) from reviews, assigning it to the variable first_row.
first_row = reviews.iloc[0,:]

In [ ]:
# Select the first 10 values from the description column in reviews, assigning the result to variable first_descriptions.
first_description = reviews.loc[0:9, 'description']

In [ ]:
# Select the records with index labels 1, 2, 3, 5, and 8, assigning the result to the variable sample_reviews.
sample_reviews = reviews.iloc[[1,2,3,5,8],:]

In [ ]:
# Create a variable df containing the country, province, region_1, and region_2 columns of the records with the index labels 0, 1, 10, and 100. 
df = reviews.loc[[0,1,10,100], ['country', 'province', 'region_1', 'region_2']]

In [ ]:
# Create a variable df containing the country and variety columns of the first 100 records.
df = reviews.loc[0:99, ['country', 'variety']]

In [ ]:
# Create a DataFrame italian_wines containing reviews of wines made in Italy. Hint: reviews.country equals what?
italian_wines = reviews[reviews.country == 'Italy']

In [ ]:
# Create a DataFrame top_oceania_wines containing all reviews with at least 95 points (out of 100) for wines from Australia or New Zealand.
top_oceania_wines = reviews[((reviews.country == 'New Zealand')|(reviews.country == 'Australia')) & (reviews.points >= 95)]

# Summary Functions and Maps

In [ ]:
# What is the median of the points column in the reviews DataFrame?
median_point = reviews.points.median()

In [ ]:
# What countries are represented in the dataset? (Your answer should not include any duplicates.)
countries = reviews.country.unique()

In [ ]:
# How often does each country appear in the dataset? Create a Series reviews_per_country mapping countries to the count of reviews of wines from that country.
reviews_per_country = reviews.country.value_counts()

In [ ]:
# Create variable centered_price containing a version of the price column with the mean price subtracted.
# (Note: this 'centering' transformation is a common preprocessing step before applying various machine learning algorithms.)
centered_price = reviews.price - reviews.price.mean()

In [ ]:
# I'm an economical wine buyer. Which wine is the "best bargain"? Create a variable bargain_wine with the title of the wine with the highest points-to-price ratio in the dataset.
reviews['points_to_price'] = reviews.points / reviews.price
idmax = reviews.points_to_price.idxmax()
bargain_wine = reviews.title.iloc[idmax]

In [ ]:
# There are only so many words you can use when describing a bottle of wine. Is a wine more likely to be "tropical" or "fruity"? Create a Series descriptor_counts counting how many times each of these two words appears in the description column in the dataset. (For simplicity, let's ignore the capitalized versions of these words.)
tropical_count = reviews.description.map(lambda p: 'tropical' in p).sum()
fruity_count = reviews.description.map(lambda p: 'fruity' in p).sum()
descriptor_count = [tropical_count, fruity_count]

In [ ]:
# We'd like to host these wine reviews on our website, but a rating system ranging from 80 to 100 points is too hard to understand - we'd like to translate them into simple star ratings. A score of 95 or higher counts as 3 stars, a score of at least 85 but less than 95 is 2 stars. Any other score is 1 star.
# Also, the Canadian Vintners Association bought a lot of ads on the site, so any wines from Canada should automatically get 3 stars, regardless of points.
# Create a series star_ratings with the number of stars corresponding to each review in the dataset.
def nb_stars(row):
    country = row.country
    points = row.points
    if country == 'Canada':
        return 3
    elif points >= 95:
        return 3
    elif (points >= 85) & (points < 95):
        return 2
    else:
        return 1

star_ratings = reviews.apply(nb_stars, axis=1)

# Grouping and Sorting

In [ ]:
# Who are the most common wine reviewers in the dataset? Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote.
taster_twitter_handle = reviews.groupby('taster_twitter_handle').taster_twitter_handle.count()

In [ ]:
# What is the best wine I can buy for a given amount of money? Create a Series whose index is wine prices and whose values is the maximum number of points a wine costing that much was given in a review. Sort the values by price, ascending (so that 4.0 dollars is at the top and 3300.0 dollars is at the bottom).
best_rating_per_price = reviews.groupby('price')['points'].max().sort_index()

In [ ]:
# What are the minimum and maximum prices for each variety of wine? Create a DataFrame whose index is the variety category from the dataset and whose values are the min and max values thereof.
price_extremes = reviews.groupby('variety')['price'].agg([min,max])

In [ ]:
# What are the most expensive wine varieties? Create a variable sorted_varieties containing a copy of the dataframe from the previous question where varieties are sorted in descending order based on minimum price, then on maximum price (to break ties).
sorted_varieties = price_extremes.sort_values(['min', 'max'], ascending=[False, False])

In [ ]:
# Create a `Series` whose index is reviewers and whose values is the average review score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.
reviewer_mean_ratings = reviews.groupby('taster_name').points.mean()

In [ ]:
reviewer_mean_ratings.describe()

In [ ]:
# What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs. For example, a pinot noir produced in the US should map to {"US", "Pinot Noir"}. Sort the values in the Series in descending order based on wine count.
country_variety_counts = reviews.groupby(['country', 'variety']).size().sort_values(ascending=False)

# Data Types and Missing Values

In [ ]:
# What is the data type of the points column in the dataset?
dtype = reviews.points.dtype

In [ ]:
# Create a Series from entries in the points column, but convert the entries to strings. Hint: strings are str in native Python.
points_string = reviews.points.astype('str')

In [ ]:
# Sometimes the price column is null. How many reviews in the dataset are missing a price?
missing_price = reviews[reviews.price.isnull()]
n_missing_price = len(missing_price)

In [ ]:
# What are the most common wine-producing regions? Create a Series counting the number of times each value occurs in the region_1 field. This field is often missing data, so replace missing values with Unknown. Sort in descending order. Your output should look something like this:
# 
# Unknown                    21247
# Napa Valley                 4480
#                            ...  
# Bardolino Superiore            1
# Primitivo del Tarantino        1
# Name: region_1, Length: 1230, dtype: int64
reviews_per_region = reviews.region_1.fillna('Unknown').value_counts(ascending=False)

# Renaming and Combining

In [ ]:
# region_1 and region_2 are pretty uninformative names for locale columns in the dataset. Create a copy of reviews with these columns renamed to region and locale, respectively.
renamed = reviews.rename(columns={'region_1' : 'region', 'region_2' : 'locale'})

In [ ]:
# Set the index name in the dataset to wines.
reindexed = reviews.rename_axis('wines', axis='rows')

In [ ]:
# The Things on Reddit dataset includes product links from a selection of top-ranked forums ("subreddits") on reddit.com. Run the cell below to load a dataframe of products mentioned on the /r/gaming subreddit and another dataframe for products mentioned on the r//movies subreddit.
gaming_products = pd.read_csv("data/top-things/top-things/reddits/g/gaming.csv")
gaming_products['subreddit'] = "r/gaming"
movie_products = pd.read_csv("data/top-things/top-things/reddits/m/movies.csv")
movie_products['subreddit'] = "r/movies"

# Create a DataFrame of products mentioned on either subreddit.
combined_products = pd.concat([gaming_products, movie_products])

In [ ]:
# The Powerlifting Database dataset on Kaggle includes one CSV table for powerlifting meets and a separate one for powerlifting competitors. Run the cell below to load these datasets into dataframes:
powerlifting_meets = pd.read_csv("data/powerlifting-database/meets.csv")
powerlifting_competitors = pd.read_csv("data/powerlifting-database/openpowerlifting.csv")

# Both tables include references to a MeetID, a unique key for each meet (competition) included in the database. Using this, generate a dataset combining the two tables into one.
left = powerlifting_meets.set_index('MeetID')
right = powerlifting_competitors.set_index('MeetID')
powerlifting_combined = left.join(right, lsuffix='_meets', rsuffix='competitors')